In [ ]:
import cv2
import os
import time

# ========== Arabic → English Folder Mapping ==========
letter_map = {
    "أ": "alef",
    "ب": "baa",
    "ت": "taa",
    "ث": "thaa",
    "ج": "geem",
    "ح": "haa",
    "خ": "khaa",
    "د": "dal",
    "ذ": "thal",
    "ر": "raa",
    "ز": "zaay",
    "س": "seen",
    "ش": "sheen",
    "ص": "saad",
    "ض": "daad",
    "ط": "taa2",
    "ظ": "thaa2",
    "ع": "ain",
    "غ": "ghain",
    "ف": "faa",
    "ق": "qaaf",
    "ك": "kaaf",
    "ل": "laam",
    "م": "meem",
    "ن": "noon",
    "ه": "haa2",
    "و": "waw",
    "ي": "yaa",
    "ئ" : "ya",
    "ة" : "taa3",
    "ال" : "al",
    "لا" : "laa"
}
# =====================================================

# === Choose Arabic letter here ===
arabic_letter = "لا"
# =================================

# Convert to English folder name
if arabic_letter not in letter_map:
    print("Error")
    exit()

folder_name = letter_map[arabic_letter]
folder_path = f"letters/{folder_name}"

# Settings
max_images = 80
delay = 0.3

# Create folder
os.makedirs(folder_path, exist_ok=True)

# Start camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Error: Could not open the camera.")
    exit()

count = 0

print(f"Starting capture for letter '{arabic_letter}' → folder '{folder_name}'")
print("Press 'q' to stop manually.")

while count < max_images:
    ret, frame = cap.read()
    if not ret:
        print("Error reading frame.")
        break

    img_name = f"{folder_path}/img_{count+1}.jpg"
    cv2.imwrite(img_name, frame)
    print(f"Saved: {img_name}")

    count += 1
    time.sleep(delay)

    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Stopped manually.")
        break

cap.release()
cv2.destroyAllWindows()
print("Done!")


Starting capture for letter 'لا' → folder 'laa'
Press 'q' to stop manually.
Saved: letters/laa/img_1.jpg
Saved: letters/laa/img_2.jpg
Saved: letters/laa/img_3.jpg
Saved: letters/laa/img_4.jpg
Saved: letters/laa/img_5.jpg
Saved: letters/laa/img_6.jpg
Saved: letters/laa/img_7.jpg
Saved: letters/laa/img_8.jpg
Saved: letters/laa/img_9.jpg
Saved: letters/laa/img_10.jpg
Saved: letters/laa/img_11.jpg
Saved: letters/laa/img_12.jpg
Saved: letters/laa/img_13.jpg
Saved: letters/laa/img_14.jpg
Saved: letters/laa/img_15.jpg
Saved: letters/laa/img_16.jpg
Saved: letters/laa/img_17.jpg
Saved: letters/laa/img_18.jpg
Saved: letters/laa/img_19.jpg
Saved: letters/laa/img_20.jpg
Saved: letters/laa/img_21.jpg
Saved: letters/laa/img_22.jpg
Saved: letters/laa/img_23.jpg
Saved: letters/laa/img_24.jpg
Saved: letters/laa/img_25.jpg
Saved: letters/laa/img_26.jpg
Saved: letters/laa/img_27.jpg
Saved: letters/laa/img_28.jpg
Saved: letters/laa/img_29.jpg
Saved: letters/laa/img_30.jpg
Saved: letters/laa/img_31.jpg
Sav

In [ ]:
import os
from pathlib import Path
import csv
from PIL import Image
import numpy as np
import mediapipe as mp
import pandas as pd

# ------------ SETTINGS ------------
ROOT_FOLDER = "letters"                  # the folder contains the folders of the letters photos
OUT_CSV = "landmarks_dataset.csv"
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}
# -----------------------------------

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

rows = []
skipped = 0
processed = 0

root = Path(ROOT_FOLDER)
if not root.exists():
    print(f"Error: folder '{ROOT_FOLDER}' not exist.")
    exit(1)

# Prepare CSV header: x0,y0,x1,y1,...,x20,y20,label
header = []
for i in range(21):
    header += [f"x{i}", f"y{i}"]
header.append("label")

# Iterate folders (each folder name is the arabic label)
for folder in sorted(root.iterdir()):
    if not folder.is_dir():
        continue
    arabic_label = folder.name
    print(f"\nProcessing folder: {arabic_label} -> {folder}")

    # Iterate images in folder
    for img_path in sorted(folder.iterdir()):
        if img_path.suffix.lower() not in IMAGE_EXTS:
            continue
        try:
            # Read image via PIL to avoid path/encoding issues
            pil_img = Image.open(img_path).convert("RGB")
            img_np = np.array(pil_img)  # RGB numpy array

            # MediaPipe expects RGB images
            results = hands.process(img_np)

            if not results.multi_hand_landmarks:
                skipped += 1
                print(f"  [skip] no hand detected: {img_path.name}")
                continue

            lm = results.multi_hand_landmarks[0].landmark  # أول يد
            # reference (wrist) coordinates
            x0 = lm[0].x
            y0 = lm[0].y

            feat = []
            # For each landmark, append (x_i - x0), (y_i - y0)
            for i in range(21):
                xi = lm[i].x
                yi = lm[i].y
                
                if i == 0:
                    feat.append(xi)
                    feat.append(yi)
                    continue
                
                feat.append(xi - x0)
                feat.append(yi - y0)

            # Append label (arabic folder name)
            rows.append(feat + [arabic_label])
            processed += 1

        except Exception as e:
            skipped += 1
            print(f"  [error] {img_path.name} -> {e}")
            continue

# Save to CSV with utf-8-sig (good for Excel)
if rows:
    df = pd.DataFrame(rows, columns=header)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\nSaved {len(df)} rows to '{OUT_CSV}' (encoding utf-8-sig).")
else:
    print("\nNo rows to save.")

print(f"Processed: {processed}, Skipped/Error: {skipped}")

hands.close()



Processing folder: .ipynb_checkpoints -> letters\.ipynb_checkpoints

Processing folder: أ -> letters\أ

Processing folder: ال -> letters\ال

Processing folder: ب -> letters\ب

Processing folder: ة -> letters\ة

Processing folder: ت -> letters\ت

Processing folder: ث -> letters\ث

Processing folder: ج -> letters\ج

Processing folder: ح -> letters\ح
  [skip] no hand detected: img_28.jpg
  [skip] no hand detected: img_33.jpg

Processing folder: خ -> letters\خ
  [skip] no hand detected: img_37.jpg
  [skip] no hand detected: img_47.jpg
  [skip] no hand detected: img_48.jpg

Processing folder: د -> letters\د

Processing folder: ذ -> letters\ذ

Processing folder: ر -> letters\ر

Processing folder: ز -> letters\ز

Processing folder: س -> letters\س

Processing folder: ش -> letters\ش

Processing folder: ص -> letters\ص

Processing folder: ض -> letters\ض

Processing folder: ط -> letters\ط

Processing folder: ظ -> letters\ظ

Processing folder: ع -> letters\ع

Processing folder: غ -> letters\غ

Pro

In [2]:
import os
from pathlib import Path
import csv
from PIL import Image
import numpy as np
import mediapipe as mp
import pandas as pd

# ------------ SETTINGS ------------
ROOT_FOLDER = "letters"                  
OUT_CSV = "landmarks_dataset.csv"        
IMAGE_EXTS = {".jpg", ".jpeg", ".png"}   
# -----------------------------------

mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

rows = []
skipped = 0
processed = 0

root = Path(ROOT_FOLDER)
if not root.exists():
    print(f"Error: المجلد '{ROOT_FOLDER}' غير موجود.")
    exit(1)

# Prepare CSV header: x0,y0,z0,x1,y1,z1,...,x20,y20,z20,label
header = []
for i in range(21):
    header += [f"x{i}", f"y{i}", f"z{i}"]
header.append("label")

# Iterate folders (each folder name is the arabic label)
for folder in sorted(root.iterdir()):
    if not folder.is_dir():
        continue
    arabic_label = folder.name
    print(f"\nProcessing folder: {arabic_label} -> {folder}")

    for img_path in sorted(folder.iterdir()):
        if img_path.suffix.lower() not in IMAGE_EXTS:
            continue
        try:
            pil_img = Image.open(img_path).convert("RGB")
            img_np = np.array(pil_img)

            results = hands.process(img_np)

            if not results.multi_hand_landmarks:
                skipped += 1
                print(f"  [skip] no hand detected: {img_path.name}")
                continue

            lm = results.multi_hand_landmarks[0].landmark

            # reference (wrist)
            x0 = lm[0].x
            y0 = lm[0].y
            z0 = lm[0].z

            feat = []

            for i in range(21):
                xi = lm[i].x
                yi = lm[i].y
                zi = lm[i].z

                if i == 0:
                    # wrist: keep raw values
                    feat.append(xi)
                    feat.append(yi)
                    feat.append(zi)
                    continue

                # subtract wrist
                feat.append(xi - x0)
                feat.append(yi - y0)
                feat.append(zi - z0)

            rows.append(feat + [arabic_label])
            processed += 1

        except Exception as e:
            skipped += 1
            print(f"  [error] {img_path.name} -> {e}")
            continue

if rows:
    df = pd.DataFrame(rows, columns=header)
    df.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
    print(f"\nSaved {len(df)} rows to '{OUT_CSV}'.")
else:
    print("\nNo rows to save.")

print(f"Processed: {processed}, Skipped/Error: {skipped}")

hands.close()



Processing folder: .ipynb_checkpoints -> letters\.ipynb_checkpoints

Processing folder: أ -> letters\أ

Processing folder: ال -> letters\ال

Processing folder: ب -> letters\ب

Processing folder: ة -> letters\ة

Processing folder: ت -> letters\ت

Processing folder: ث -> letters\ث

Processing folder: ج -> letters\ج

Processing folder: ح -> letters\ح
  [skip] no hand detected: img_28.jpg
  [skip] no hand detected: img_33.jpg

Processing folder: خ -> letters\خ
  [skip] no hand detected: img_37.jpg
  [skip] no hand detected: img_47.jpg
  [skip] no hand detected: img_48.jpg

Processing folder: د -> letters\د

Processing folder: ذ -> letters\ذ

Processing folder: ر -> letters\ر

Processing folder: ز -> letters\ز

Processing folder: س -> letters\س

Processing folder: ش -> letters\ش

Processing folder: ص -> letters\ص

Processing folder: ض -> letters\ض

Processing folder: ط -> letters\ط

Processing folder: ظ -> letters\ظ

Processing folder: ع -> letters\ع

Processing folder: غ -> letters\غ

Pro